In [1]:
2 + 2

4

In [3]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [4]:
load_dotenv()
api_key=os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please look for the issue and fix it")
elif api_key[:8]!="sk-proj-":
    print("An api key was found but it is not starting with the correct syntax sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found and it looks like it might have space or tab characters ar the start or enf - please remove them")
else:
    print("API key found we are good to go!!!")
    


API key found we are good to go!!!


In [5]:
openai= OpenAI()

In [10]:
# To give you a preview -- calling OpenAI with these messages is this easy.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to chat with you. How can I assist you today?


In [7]:
# A class to represent a webpage

class Website:
    """
    A utility class to represtn a website that we have scraped
    """
    url: str
    title: str
    text: str
    
    def __init__(self, url):
        """Create this website object from the given url using the Beautifulsoup Library
        
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title= soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [9]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [ ]:
# ## Types of prompts
# You may know this already - but if not, you will get very familiar with it!
# Models like GPT4o have been trained to receive instructions in a particular way.
# They expect to receive:
# **A system prompt** that tells them what task they are performing and what tone they should use
# **A user prompt** -- the conversation starter that they should reply to

In [11]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [18]:
# A function that write user prompt and ask for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titles {website.title} \n "
    user_prompt += "The content of this website is as follows; \
         \n please provide a short summary of this website in markdown. \
         \n If it includes news or announcements, then summarize these too. \n\n"
    user_prompt += website.text
    return user_prompt

In [19]:
print(user_prompt_for(ed))

You are looking at a website titles Home - Edward Donner 
 The content of this website is as follows;          
 please provide a short summary of this website in markdown.          
 If it includes news or announcements, then summarize these too. 

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of 

In [20]:
def message_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [21]:
message_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titles Home - Edward Donner \n The content of this website is as follows;          \n please provide a short summary of this website in markdown.          \n If it includes news or announcements, then summarize these too. \n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a

In [22]:
# And now calling OpenAI API. In the standard way

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model= "gpt-4o-mini",
        messages=message_for(website)
    )
    return response.choices[0].message.content

In [23]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website serves as a hub for his interests and professional endeavors related to coding and large language models (LLMs). He identifies himself as the co-founder and CTO of Nebula.io, a company that leverages AI to enhance talent discovery and management. Donner has a background as the founder and CEO of the AI startup untapt, which was acquired in 2021.\n\nAdditionally, his interests include DJing, amateur electronic music production, and engaging with the tech community through platforms like Hacker News.\n\n## Recent Announcements\n1. **May 28, 2025** - Introduction of courses aimed at helping individuals become experts and leaders in the field of LLMs.\n2. **May 18, 2025** - Announcement of the 2025 AI Executive Briefing.\n3. **April 21, 2025** - Launch of "The Complete Agentic AI Engineering Course."\n4. **January 23, 2025** - Hosting of an LLM workshop, providing hands-on resources related to agents. \n\nThe website invite

In [24]:
# A function to dispaly this nicely, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [25]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

The website belongs to Ed Donner, who has a keen interest in coding, experimenting with Large Language Models (LLMs), and DJing. He is the co-founder and CTO of Nebula.io, a company focused on leveraging AI to help individuals uncover their potential and connect with suitable job opportunities. Ed has previously founded the AI startup untapt, which was acquired in 2021. 

### Notable Announcements
- **May 28, 2025**: Announcement regarding connecting courses to become an LLM expert and leader.
- **May 18, 2025**: Information about the 2025 AI Executive Briefing.
- **April 21, 2025**: Launch of the "Complete Agentic AI Engineering Course."
- **January 23, 2025**: Availability of resources for the "LLM Workshop – Hands-on with Agents." 

The site reflects Ed's passion for technology, particularly in AI, and encourages visitors to connect with him for further engagement.

In [26]:
display_summary("https://anthropic.com")

# Anthropic Summary

Anthropic is focused on creating AI systems, specifically through their model, Claude, which emphasizes safety and alignment with human values. The site highlights the various versions of Claude, including the latest models: **Claude Opus 4** and **Claude Sonnet 4**, which are heralded as the most advanced offerings yet. 

## Key Features
- **Claude Capabilities**: Anthropic positions Claude as an intelligent model that can power applications, coding, and various solutions across industries like education and financial services.
- **Research Commitments**: The website emphasizes responsible AI development, featuring research initiatives, safety policies, and transparency efforts.
- **User Resources**: It provides access to training materials and customer stories to inspire users in leveraging Claude effectively.

## Recent Announcements
- Introduction of **Claude Opus 4**, noted as the most intelligent AI model available.
- Emphasis on possible societal impacts and alignment challenges related to AI development.

Overall, Anthropic is committed to the responsible creation and deployment of AI technologies, firmly putting safety and human benefits at the forefront of their operations.